In [1]:
from googlesearch import search

In [5]:
import warnings
warnings.filterwarnings("ignore")
from langchain.document_loaders import TextLoader , UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from newspaper import Article
from sentence_transformers import SentenceTransformer
import faiss
import os
import google.generativeai as genai
from security import MY_API_KEY


#URLS = input("Enter thearticle URLS: ").split()

class quesitionChatBot:

    def __init__(self , chunk_size=1000):

        genai.configure(api_key = MY_API_KEY)

        self.model = genai.GenerativeModel(model_name = "gemini-1.5-flash")
        self.encoder = SentenceTransformer("all-mpnet-base-v2")
        self.splitter = RecursiveCharacterTextSplitter(chunk_size =chunk_size , chunk_overlap = 200)
        self.index = faiss.IndexFlatL2()
        self.vectors = []
        self.texts = []
        self.chunks = []

        self.prompt = "Use the following portion of a long document to see if any of the text    is proper the question. \nReturn relevent text varbit.\n"




    def getTextFromURLS(self,URLS):

        self.texts = []
        for url in URLS:

            try:
                article = Article(url)
                article.download()
                article.parse()
                if(len(article.text) >100):
                    self.texts.append(article.text[:1000])
                
                
            except:
                pass
        
        return self.textToChunks()
    
    
    def getTextFromPDF(self,path):

        loader = UnstructuredPDFLoader(file_path = r""+path)
        text = loader.load()
        self.texts = [text[0].page_content]
        return self.textToChunks()
    
    def textToChunks(self):

        self.chunks = []

        for text in self.texts:
            self.chunks =   self.chunks + self.splitter.split_text(text)
        
        self.chunksToVectorsAndIndex()
        return len(self.chunks)
    
    def chunksToVectorsAndIndex(self):

        self.vectors = self.encoder.encode(self.chunks)
        self.index = faiss.IndexFlatL2( self.vectors.shape[1])
        self.index.add(self.vectors)
    
    def askQuestion(self , question):
        vec = self.encoder.encode([question])
        _ , idx = self.index.search(vec , k = 4)
        resopnces = ""

        for i in idx[0]:
            resopnces+= "\n" +( self.model.generate_content( f"{self.prompt + self.chunks[i] }\nquestion: {question}\n Relevent text if any: ").text)
        
        final_output = self.model.generate_content( f" {self.prompt}\n{resopnces}\n question: {question}\n Relevent text if any: ").text
        
        return  final_output


if __name__ == "__main__":


    query = input("Enter the search ")
    inp = ""
    for web in search(query , num=6, stop=6, pause=2):
        inp += " " + web
    cb = quesitionChatBot()

    if(inp.strip()[-4:] == ".pdf"):
        cb.getTextFromPDF(inp.strip())
    else: 
        cb.getTextFromURLS(inp.split())

    question = ""
    while(question != "quit"):
        question = input("Enter the question : ")
        print(cb.askQuestion(question))

HTTPError: HTTP Error 429: Too Many Requests

In [37]:
import google.generativeai as genai
import re
from security import MY_API_KEY

genai.configure(api_key = MY_API_KEY)

model =  genai.GenerativeModel(model_name = "gemini-1.5-flash")

In [95]:
fromm = "new york"
to = "King of Prussi"
days_count = 4
places =["Rta Cabinet Store", "Avalon Flooring", "Bagelicious-King Of Prussia"]

format = """ {
    "Day-1": {
        "1" : "Visit the Statue of Liberty",
        "2" : "Breakfast at a local diner",
        "3" : "Explore Central Park",
        "5" : "Lunch at a food truck",
        "6" : "Visit the Metropolitan Museum of Art",
        "7" : "Dinner at an iconic New York pizzeria",
        "8" : "Return to hotel"
    },
    "Day-2": {
        "1" : "Walk the High Line",
        "2" : "Brunch at a rooftop restaurant",
        "3" : "Take a ferry to Staten Island",
        "5" : "Lunch at Chelsea Market",
        "6" : "Visit the Museum of Modern Art (MoMA)",
        "7" : "Dinner at a fine dining restaurant",
        "8" : "Return to hotel"
    },
    "Day-days_count": {
        "1" : "Walk across the Brooklyn Bridge",
        "2" : "Breakfast at a bagel shop",
        "3" : "Visit the American Museum of Natural History",
        "5" : "Lunch in a neighborhood café",
        "6" : "Stroll through the New York Botanical Garden",
        "7" : "Dinner at a trendy restaurant in the Meatpacking District",
        "8" : "Return to hotel"
    },
    "Cost" : "1500$" ,
    "Estimation" :[ "your content about cost" ]
}"""
query = f"(Not need to be realistic  ) I want to travel from {fromm} to {to} for  {days_count} days_count and I also want you to mention this places compulsorily places : {places}  give me a me a tour Itinerary(not need to be realistic) and cost(cost need to be realistic) of travel via flight generate the content in the format like this format : {format}"
op = model.generate_content(query).text


In [24]:
op = op.lower().replace("*","").replace("#","")
plan_start = re.search("day\s*1",op)

if(plan_start!=None):
    plan_start = plan_start.span()[0]

plan_end = re.search(f"day\s*{days_count}",op)

if(plan_end != None):
    plan_end = plan_end.span()[1]

last_day_senatance_length = 40
cost_start=None

while(cost_start==None):
    last_day_senatance_length-=1
    cost_start = re.search("cost",op[plan_end+last_day_senatance_length:])
    

if(cost_start!=None):
    cost_start = cost_start.span()[0]


plan = op[plan_start:plan_end]+op[plan_end:][:cost_start]
last_dot = plan.rfind(".")
plan = plan[:last_dot+1]

cost = op[plan_end+last_day_senatance_length:][cost_start:]
print(plan)

day 1: arrival in chennai & heritage exploration

 morning: arrive at chennai international airport (maa). transfer to your hotel in the city center.
 afternoon: explore the iconic kapaleeshwarar temple, a vibrant south indian temple dedicated to lord shiva. immerse yourself in the colorful rituals and architecture.
 evening: enjoy a delicious south indian dinner at dakshin, a renowned restaurant in the itc grand chola hotel.

day 2: cultural immersions & beachside relaxation

 morning: visit the marina beach, one of the longest urban beaches in the world. stroll along the shore, enjoy the fresh sea breeze, and soak in the vibrant atmosphere.
 afternoon: step back in time at fort st. george, a historic british fort that played a pivotal role in colonial india. explore the museum and learn about chennai's rich history.
 evening: experience the bustling parthasarathy temple, a hindu temple dedicated to lord krishna. witness the traditional evening rituals and soak in the spiritual ambian